# Colab configuration

Mount your google drive to access the files stored.

In [ ]:
%cd /content/drive/MyDrive/your_path/sqlite3-notes
%ll

Or download the file directly from github

In [2]:
!curl -O https://raw.githubusercontent.com/G0erman/sqlite3-notes/main/sqlite3.po

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 79525  100 79525    0     0   308k      0 --:--:-- --:--:-- --:--:--  308k


# Exploration

In [3]:
!tail sqlite3.po -n 30

msgstr ""
"Versiones antiguas de SQLite tienen problemas compartiendo conexiones entre "
"hilos. Es por ello que el módulo de Python no permite compartir conexiones y "
"cursores entre hilos. Si se quiere intentar esto, se obtendrá una excepción "
"en tiempo de ejecución."

#: ../Doc/library/sqlite3.rst:1104
msgid ""
"The only exception is calling the :meth:`~Connection.interrupt` method, "
"which only makes sense to call from a different thread."
msgstr ""
"La única excepción es llamando el método :meth:`~Connection.interrupt`, el "
"cual solamente tiene sentido llamarlo desde un hilo diferente."

#: ../Doc/library/sqlite3.rst:1108
msgid "Footnotes"
msgstr "Notas al pie"

#: ../Doc/library/sqlite3.rst:1109
msgid ""
"The sqlite3 module is not built with loadable extension support by default, "
"because some platforms (notably Mac OS X) have SQLite libraries which are "
"compiled without this feature. To get loadable extension support, you must "
"pass --enable-loadable-sqlite-extension

The file contains the below structure:

* `#: ..(library):line_number`
* `msgid` Original text in English.
* `msgstr` Translated text to Spanish.

To do: find a regular expression tha parse this structure.

# Libraries

In [4]:
!pip install polib

In [18]:
# Utils
import re 

# Handly polib
import polib

# Natural Language Processing
import nltk
nltk.download('stopwords')
nltk.download('punkt') # sentence tokenizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Read Data

In [6]:
po = polib.pofile('sqlite3.po')
valid_entries = [entry for entry in po if not entry.obsolete]

In [7]:
english_text = [entry.msgid for entry in valid_entries]
spanish_text = [entry.msgstr for entry in valid_entries]

# Text Processing

In [13]:
english_text[:5]

[':mod:`sqlite3` --- DB-API 2.0 interface for SQLite databases',
 '**Source code:** :source:`Lib/sqlite3/`',
 "SQLite is a C library that provides a lightweight disk-based database that doesn't require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage.  It's also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle.",
 'The sqlite3 module was written by Gerhard Häring.  It provides a SQL interface compliant with the DB-API 2.0 specification described by :pep:`249`.',
 'To use the module, you must first create a :class:`Connection` object that represents the database.  Here the data will be stored in the :file:`example.db` file::']

## Basic statistics

In [54]:
# Split sentences in words
all_text = [nltk.word_tokenize(sentence) for sentence in english_text]

In [55]:
all_text[:2]

[[':',
  'mod',
  ':',
  '`sqlite3`',
  '--',
  '-',
  'DB-API',
  '2.0',
  'interface',
  'for',
  'SQLite',
  'databases'],
 ['**Source', 'code', ':', '**', ':', 'source', ':', '`Lib/sqlite3/`']]

In [56]:
def clean_word(x):
  """Delete not alphabetic caracteres."""
  word = re.sub(r'\W+', r' ', x.lower()).strip()
  word = re.sub(r'\d+', r' ', word).strip()
  #print(word)
  if (not word) or (len(word) < 3):
    return None
  else:
    return word

def test_clean_word():
  assert(clean_word(':') == None)
  assert(clean_word('Interface') == 'interface')
  assert(clean_word('DB-API') == 'db api')
  assert(clean_word('5') == None)
  assert(clean_word(' ti') == None)

test_clean_word()


interface
db api

ti


In [66]:
all_text_clean = []
for sentence in all_text:
  for word in sentence:
    final_word = clean_word(word)
    #print(word)
    #print(final_word)
    all_text_clean.append(final_word)

#all_text_clean = [clean_word(word) for word in sentence for sentence in all_text]
#all_text_clean = [nltk.tokenize(word) for word in all_text_clean]

Streaming output truncated to the last 5000 lines.

connection isolation_level
property
of

class

connection
objects

sqlite
natively
supports
only
the
types
text

integer

real

blob
and
null

if
you
want
to
use
other
types
you
must
add
support
for
them
yourself

the
detect_types
parameter
and
the
using
custom
converters
registered
with
the
module level

func

register_converter
function
allow
you
to
easily
do
that

detect_types
defaults
to


i
e
off

no
type
detection


you
can
set
it
to
any
combination
of

const

parse_decltypes
and

const

parse_colnames
to
turn
type
detection
on

by
default

check_same_thread
is

const

true
and
only
the
creating
thread
may
use
the
connection

if
set

const

false

the
returned
connection
may
be
shared
across
multiple
threads

when
using
multiple
threads
with
the
same
connection
writing
operations
should
be
serialized
by
the
user
to
avoid
data
corruption

by
default

the

mod

sqlite
module
uses
its

class

connection
class
for
the
connect
call



In [69]:
# Exclude none
all_text_clean = [word for word in all_text_clean if word]
all_text_clean[:10]

['mod',
 'sqlite',
 'db api',
 'interface',
 'for',
 'sqlite',
 'databases',
 'source',
 'code',
 'source']

In [73]:
# all words
len(all_text_clean)

3884

In [12]:
def catch_reference(x):
  """Catch objects :mod:`sqlite3`, This could be references to: 
  modules, libraries, clases, etc.

  parameters:
    x: all text to parse

  returns:
    ref: object's dictionary 
  """
  ref = re.sub(r':(mod):`(sqlite3)`(.*)', r'{"\1":["\2"]}', x)
  print(ref)
  return ref

def test_catch_reference():
  assert catch_reference(':mod:`sqlite3` --- DB-API 2.0 int... databases') == \
  'mod,sqlite3'

test_catch_reference()  

{"mod":["sqlite3"]}


AssertionError: ignored

In [ ]:
re.findall()

TypeError: ignored